# **1. Data Recolection**

Actualmente en ***Too Good To Go*** no disponemos de un servicio de **food delivering** para poder llegar a un publico mas amplio promocionando **habitos de consumo** que **mejoren nuestra sociedad**. El ano pasado tuvimos una venta total de 121 millones de meals, sabemos que compararnos con empresas como Doordash que suponen mas de 2161 millones de delivery orders puede ser ambicio, pero aun si es posible. 

Por esta razon en este proyecto trabajaremos en el analisis de los usuarios de food ordering/delivering de las empresas de ***Doordash*** y ***UberEats*** en la red social ***Reddit***. Ambas suponen **mas del 80% del mercado**. Con esta informacion podemos profundizar en el perfil de sus clientes y repartidores. Como en cualquier otra mercado, el cliente siempre tiene la razon, pero este en particular depende de los repartidores que son la cara de tu empresa.

Problemas que podemos encontrar.
* Unicamente enfocarnos en Reddit puede influer en no tener la foto completa de que esta ocurriendo.
* Existen otras companias de food delivering que no estamos teniendo en cuenta en nuestro estudio.
* Posibilidad de no existencia de diferencias entre empresas para Reddit.




In [1]:
import praw
from datetime import datetime
import pandas as pd
import re #Regular expression operation - 
import json
import os

with open("config.json", "r") as file:
    config = json.load(file)

reddit = praw.Reddit( 
                    client_id = config["client_id"],
                    client_secret = config["client_secret"],
                    user_agent = config["user_agent"]
)

In [2]:
import time

# What subreddit are important to us
subreddit = reddit.subreddit('doordash')
#subreddit = reddit.subreddit('UberEATS')

# List to save our post
data = []

# Save the 1000 newsest posts from our reddit
for post in subreddit.new(limit=1000):  # Obtiene los 1000 posts más recientes
    data.append([
        post.created_utc,          # Fecha de creación en formato UTC
        post.title,               # Título del post
        post.selftext,            # Texto del post (si es un post de texto)
        post.subreddit.display_name,  # Nombre del subreddit
        post.num_comments,        # Número de comentarios
        post.score,               # Puntos de la publicación (score)
        post.upvote_ratio,        # Ratio de upvotes
        post.url,                 # URL del post
        post.is_self,             # Si el post es de tipo "self post"
        post.link_flair_text,     # Flair del post (puede ser None si no tiene flair)
        post.author.name if post.author else None,  # Autor del post
        post.num_reports,         # Número de reportes
        post.distinguished,       # Si el post es distinguido (sticky, gold, etc.)
        post.over_18,             # Si es un post NSFW
    ])


# Transform to a DataFrame
df = pd.DataFrame(data, columns=[
    'created_utc', 'title', 'self_text', 'subreddit', 'num_comments', 
    'score', 'upvote_ratio', 'url', 'is_self', 'link_flair_text', 
    'author', 'num_reports', 'distinguished', 'over_18'
])


# Set date for our CSV creation
current_time = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')

# Save the CSV with date info
file_name = f'subreddit_new_posts_{current_time}.csv'
df.to_csv(file_name, index=False)

In [3]:
# Select all the files create for our analysis
files = os.listdir('Datasets')
csv_files = [f for f in files if f.endswith(".csv")]

In [4]:
dataframes = []

# Import files from CSV with pandas on the same list of dataframes 
# Create new vars to catalogue every record from every archive
for file in csv_files:
    file_path = os.path.join('Datasets', file)
    df = pd.read_csv(file_path)


    
    if len(file.split("_"))==5:
        df['file_date']=file.split("_")[3]
        df['file_hour']=file.split("_")[4]        
        df['file_type']=file.split("_")[1]
    else:
        df['file_date']=file.split("_")[2]
        df['file_hour']=file.split("_")[3]        
        df['file_type']='new'
    dataframes.append(df)


In [5]:


# Verifica el contenido del primer dataframe
combined_df = pd.concat(dataframes, ignore_index=True)
#combined_df.columns
#combined_df[(combined_df['created_utc.1'] != combined_df['created_utc']) & combined_df['created_utc.1'].notnull()]
combined_df=combined_df.drop(columns='created_utc.1')

In [6]:
combined_df.head(3)

,created_utc,title,self_text,subreddit,num_comments,score,upvote_ratio,url,file_date,file_hour,file_type,is_self,link_flair_text,author,num_reports,distinguished,over_18
0,1.731987e+09,First time driving tonight ... Is this normal???,I drove for 2.5 hours doing Uber Eats for the ...,UberEATS,2,1,1.0,https://www.reddit.com/r/UberEATS/comments/1gu...,2024-11-18,19-38-10.csv,new,NaN,NaN,NaN,NaN,NaN,NaN
1,1.731983e+09,Referral rewards no longer working on pickup o...,Referral rewards used to work on pickup buti c...,UberEATS,1,1,1.0,https://www.reddit.com/r/UberEATS/comments/1gu...,2024-11-18,19-38-10.csv,new,NaN,NaN,NaN,NaN,NaN,NaN
2,1.731980e+09,Does Ubereats even have support,"I called the support number, tried livechat, s...",UberEATS,2,0,0.5,https://www.reddit.com/r/UberEATS/comments/1gu...,2024-11-18,19-38-10.csv,new,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
combined_df=combined_df.sort_values(by=['file_date','file_hour'],ascending=False)

In [8]:
unique_rows = combined_df.drop_duplicates(subset=['created_utc', 'title', 'self_text', 'subreddit','author'])

In [9]:
unique_rows = unique_rows.drop_duplicates(subset=['title','self_text','author'])

In [10]:
unique_rows = combined_df.drop_duplicates(subset=['created_utc', 'title'])

In [11]:
#2687
#unique_rows[unique_rows.duplicated(subset=['title'],keep=False)].sort_values(by=['title'],ascending=False)
unique_rows.isnull().sum()


created_utc           0
title                 0
self_text           367
subreddit             0
num_comments          0
score                 0
upvote_ratio          0
url                   0
file_date             0
file_hour             0
file_type             0
is_self             328
link_flair_text    2371
author              348
num_reports        2727
distinguished      2724
over_18             328
dtype: int64

In [12]:
unique_rows=unique_rows.drop(columns=['num_reports','distinguished','over_18'])

In [13]:
unique_rows['info_url']= unique_rows['url'].apply(lambda x: x.split('/')[-1] 
                             if not x.endswith('/') 
                             else None
                                                 )

In [14]:
#info_url has a media attached
#the selftext has a media description(photo...)
unique_rows['media'] = unique_rows['info_url'].apply(lambda x: 'y' 
                              if x is not None 
                              else 'n')

In [15]:
unique_rows.isnull().sum()

created_utc           0
title                 0
self_text           367
subreddit             0
num_comments          0
score                 0
upvote_ratio          0
url                   0
file_date             0
file_hour             0
file_type             0
is_self             328
link_flair_text    2371
author              348
info_url           1709
media                 0
dtype: int64

In [18]:
unique_rows.to_csv('post_reddit.csv', index=False)